# 1. IMPORT LIBRARIES

In [1]:
import cloudscraper # Using cloudscraper instead of requests to overcome cloudflare issues
scraper = cloudscraper.create_scraper()
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re

# 2. SCRAPE NECESSARY INFORMATION

In [2]:
# Function for finding all the required information for every car on the page (minus the sponsored ad which...)
# is always the first of the page

def items_page(url):
    page = scraper.get(url)
    soup = BeautifulSoup(page.content, "html.parser") 
    return soup.find_all("div", class_="product-card-content__car-info")[1:]

In [3]:
# Function for returning the URL to the next page

def return_next_page(url):
    page = scraper.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    return soup.find("a", class_="pagination--right__active")["href"]

In [19]:
# Function to get all of the necessary details for every car

def get_details(make_and_model, year, mileage, engine_size, transmission, fuel_type, price, value_indicator):
    for car in items:

        # VALUE INDICATOR
        try:
            car_value = car.find("li", class_="badge-group__item").text.strip()
            if re.match("Lower price|Great price|Good price|Fair price|Higher price", car_value): # check an indicated value is given and not another potential tag such as "finance available"
                value_indicator.append(car_value.split(" ")[0]) # if given, append to list
            else: # if no value is given, continue to next car (same for all below)
                continue
        except:
            continue

        # MAKE AND MODEL
        try:
            make_and_model.append(car.find("h3", class_= "product-card-details__title").text.strip())
        except:
            continue

        # PRICE
        try:
            price.append(re.sub("\D", "", car.find("div", class_="product-card-pricing__price").text.strip())) # keep only numbers
        except:
            continue

        # Following extracted from the code contained within:
        try:
            key_specs = car.find_all("li", class_= "atc-type-picanto--medium")
        except:
            continue
        # YEAR
        try:
            car_year = (key_specs[0].text.split(" ")[0])
            if re.match("^20[0-9]{2}$", car_year): # checking it is a year that is provided
                year.append(car_year)
            else:
                year.append("NaN") # if no regex match, print "NaN" (same for all of the following key specs)
        except:
            continue

        # MILEAGE
        try:
            car_mileage = (key_specs[2].text)
            if re.match("(?i)(.*)(?=mile)", car_mileage): # case insensitive check for ending 'mile'
                mileage.append(re.sub("\D", "", car_mileage)) # if a match, append just the numbers
            else:
                mileage.append("NaN") 
        except:
            continue

        # ENGINE SIZE
        try:
            car_engine_size = (key_specs[3].text)
            if re.match("(?i)(.*)(?=L)", car_engine_size): # case insensitive check for ending "L"
                engine_size.append(re.sub("L", "", car_engine_size)) #if a match, append the engine size minus "L"
            else:
                engine_size.append("NaN")
        except:
            continue

        # TRANSMISSION
        try:
            car_transmission = (key_specs[5].text)
            if re.match("(?i)Automatic|Manual", car_transmission): # case insensitive check for transmission
                transmission.append(car_transmission) # append if a match
            else:
                transmission.append("NaN")
        except:
            continue

        # FUEL TYPE - will include petrol and petrol hybrid cars and then also diesel and electric
        try:
            car_fuel_type = (key_specs[6].text)
            if re.match("(?i)Petrol|^Diesel$|Electric", car_fuel_type): # case insensitive check for any form of petrol (including hybrids), but only straight diesel, and electric
                fuel_type.append(car_fuel_type) # apppend if a match
            else:
                fuel_type.append("NaN")
        except IndexError:
            fuel_type.append("NaN") # was an occassion of [6] being out of index range, thus if this is the case, fill with NaN


In [20]:
# The first page of every year in the range 2006-2020 with the refined search
base_URLs = ["https://www.autotrader.co.uk/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2006&year-to=2006&exclude-writeoff-categories=on&page=1",
             "https://www.autotrader.co.uk/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2007&year-to=2007&exclude-writeoff-categories=on&page=1",
             "https://www.autotrader.co.uk/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2008&year-to=2008&exclude-writeoff-categories=on&page=1",
             "https://www.autotrader.co.uk/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2009&year-to=2009&exclude-writeoff-categories=on&page=1",
             "https://www.autotrader.co.uk/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2010&year-to=2010&exclude-writeoff-categories=on&page=1",
             "https://www.autotrader.co.uk/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2011&year-to=2011&exclude-writeoff-categories=on&page=1",
             "https://www.autotrader.co.uk/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2012&year-to=2012&exclude-writeoff-categories=on&page=1",
             "https://www.autotrader.co.uk/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2013&year-to=2013&exclude-writeoff-categories=on&page=1",
             "https://www.autotrader.co.uk/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2014&year-to=2014&exclude-writeoff-categories=on&page=1",
             "https://www.autotrader.co.uk/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2015&year-to=2015&exclude-writeoff-categories=on&page=1",
             "https://www.autotrader.co.uk/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2016&year-to=2016&exclude-writeoff-categories=on&page=1",
             "https://www.autotrader.co.uk/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2017&year-to=2017&exclude-writeoff-categories=on&page=1",
             "https://www.autotrader.co.uk/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2018&year-to=2018&exclude-writeoff-categories=on&page=1",
             "https://www.autotrader.co.uk/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2019&year-to=2019&exclude-writeoff-categories=on&page=1",
             "https://www.autotrader.co.uk/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2020&year-to=2020&exclude-writeoff-categories=on&page=1"]

# Empty lists
make_and_model = []
year = []
mileage = []
engine_size = []
transmission = []
fuel_type = []
price = []
value_indicator = []

for first_page_year_URL in base_URLs: # for loop to start at the first page of every year
    total_year_URLs = [first_page_year_URL]
    
    while (len(total_year_URLs) < 176): # set the number of pages 
        items = items_page(total_year_URLs[-1]) # run function to get all necessary information from the page to then be scraped
        print(total_year_URLs[-1]) # print the URL that is being scraped to keep track of progress
        get_details(make_and_model, year, mileage, engine_size, transmission, fuel_type, price, value_indicator) #run function
        next_page_url = return_next_page(total_year_URLs[-1]) # run the function to get the next page from the last in the list of the year
        total_year_URLs.append(next_page_url) # append the next page's URL to the year's total URLs
        

https://www.autotrader.co.uk/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2006&year-to=2006&exclude-writeoff-categories=on&page=1
https://www.autotrader.co.uk:443/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2006&year-to=2006&exclude-writeoff-categories=on&page=2
https://www.autotrader.co.uk:443/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2006&year-to=2006&exclude-writeoff-categories=on&page=3
https://www.autotrader.co.uk:443/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2006&year-to=2006&exclude-writeoff-categories=on&page=4
https://www.autotrader.co.uk:443/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2006&

https://www.autotrader.co.uk:443/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2006&year-to=2006&exclude-writeoff-categories=on&page=40
https://www.autotrader.co.uk:443/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2006&year-to=2006&exclude-writeoff-categories=on&page=41
https://www.autotrader.co.uk:443/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2006&year-to=2006&exclude-writeoff-categories=on&page=42
https://www.autotrader.co.uk:443/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2006&year-to=2006&exclude-writeoff-categories=on&page=43
https://www.autotrader.co.uk:443/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-fr

https://www.autotrader.co.uk:443/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2006&year-to=2006&exclude-writeoff-categories=on&page=79
https://www.autotrader.co.uk:443/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2006&year-to=2006&exclude-writeoff-categories=on&page=80
https://www.autotrader.co.uk:443/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2006&year-to=2006&exclude-writeoff-categories=on&page=81
https://www.autotrader.co.uk:443/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2006&year-to=2006&exclude-writeoff-categories=on&page=82
https://www.autotrader.co.uk:443/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-fr

https://www.autotrader.co.uk:443/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2006&year-to=2006&exclude-writeoff-categories=on&page=118
https://www.autotrader.co.uk:443/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2006&year-to=2006&exclude-writeoff-categories=on&page=119
https://www.autotrader.co.uk:443/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2006&year-to=2006&exclude-writeoff-categories=on&page=120
https://www.autotrader.co.uk:443/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2006&year-to=2006&exclude-writeoff-categories=on&page=121
https://www.autotrader.co.uk:443/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&yea

https://www.autotrader.co.uk:443/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2006&year-to=2006&exclude-writeoff-categories=on&page=157
https://www.autotrader.co.uk:443/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2006&year-to=2006&exclude-writeoff-categories=on&page=158
https://www.autotrader.co.uk:443/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2006&year-to=2006&exclude-writeoff-categories=on&page=159
https://www.autotrader.co.uk:443/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2006&year-to=2006&exclude-writeoff-categories=on&page=160
https://www.autotrader.co.uk:443/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&yea

...

https://www.autotrader.co.uk:443/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2020&year-to=2020&exclude-writeoff-categories=on&page=8
https://www.autotrader.co.uk:443/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2020&year-to=2020&exclude-writeoff-categories=on&page=9
https://www.autotrader.co.uk:443/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2020&year-to=2020&exclude-writeoff-categories=on&page=10
https://www.autotrader.co.uk:443/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2020&year-to=2020&exclude-writeoff-categories=on&page=11
https://www.autotrader.co.uk:443/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from

https://www.autotrader.co.uk:443/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2020&year-to=2020&exclude-writeoff-categories=on&page=47
https://www.autotrader.co.uk:443/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2020&year-to=2020&exclude-writeoff-categories=on&page=48
https://www.autotrader.co.uk:443/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2020&year-to=2020&exclude-writeoff-categories=on&page=49
https://www.autotrader.co.uk:443/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2020&year-to=2020&exclude-writeoff-categories=on&page=50
https://www.autotrader.co.uk:443/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-fr

https://www.autotrader.co.uk:443/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2020&year-to=2020&exclude-writeoff-categories=on&page=86
https://www.autotrader.co.uk:443/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2020&year-to=2020&exclude-writeoff-categories=on&page=87
https://www.autotrader.co.uk:443/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2020&year-to=2020&exclude-writeoff-categories=on&page=88
https://www.autotrader.co.uk:443/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2020&year-to=2020&exclude-writeoff-categories=on&page=89
https://www.autotrader.co.uk:443/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-fr

https://www.autotrader.co.uk:443/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2020&year-to=2020&exclude-writeoff-categories=on&page=125
https://www.autotrader.co.uk:443/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2020&year-to=2020&exclude-writeoff-categories=on&page=126
https://www.autotrader.co.uk:443/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2020&year-to=2020&exclude-writeoff-categories=on&page=127
https://www.autotrader.co.uk:443/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2020&year-to=2020&exclude-writeoff-categories=on&page=128
https://www.autotrader.co.uk:443/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&yea

https://www.autotrader.co.uk:443/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2020&year-to=2020&exclude-writeoff-categories=on&page=164
https://www.autotrader.co.uk:443/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2020&year-to=2020&exclude-writeoff-categories=on&page=165
https://www.autotrader.co.uk:443/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2020&year-to=2020&exclude-writeoff-categories=on&page=166
https://www.autotrader.co.uk:443/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2020&year-to=2020&exclude-writeoff-categories=on&page=167
https://www.autotrader.co.uk:443/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&yea

In [21]:
#putting the scraped information into a dataframe

car_details = pd.DataFrame({"Make and Model": make_and_model, "Year": year, "Mileage": mileage, "Transmission": transmission, "Engine Size (L)": engine_size, "Fuel Type": fuel_type, "Price (£)": price, "Indicated Value": value_indicator}) 
print("Length of dataframe:",(len(car_details)))
car_details[:10]

Length of dataframe: 16608


,Make and Model,Year,Mileage,Transmission,Engine Size (L),Fuel Type,Price (£),Indicated Value
0,Renault Clio,2006,55378,Manual,1.6,Petrol,1675,Great
1,SEAT Leon,2006,110988,Manual,2.0,Petrol,2350,Great
2,MINI Hatch Cooper,2006,69000,Manual,1.6,Petrol,2499,Fair
3,Ford Fiesta,2006,85714,Manual,1.6,Diesel,1699,Great
4,Suzuki Grand Vitara,2006,77000,Manual,1.6,Petrol,2450,Great
5,BMW 3 Series,2006,81500,Manual,2.0,Petrol,1990,Lower
6,MINI Hatch,2006,85000,Manual,1.6,Petrol,1795,Fair
7,Toyota Yaris,2006,99000,Manual,1.3,Petrol,1595,Great
8,Vauxhall Corsa,2006,60000,Manual,1.2,Petrol,1650,Great
9,Suzuki Grand Vitara,2006,77000,Manual,1.6,Petrol,2450,Great


# 3. CLEANING THE DATA

### 3.1. Dropping duplicates

In [22]:
# Create a copy for the following cleaning
car_details_clean = car_details.copy(deep=True)

# Duplicates can be present due to sellers paying to advertise their car more

# checking that there are duplicates
print(car_details_clean.value_counts()[:3], "\n") 

# drop duplicates and see updated dataframe
car_details_clean = car_details_clean.drop_duplicates()

# ensure duplicates have been removed
print(car_details_clean.value_counts()[:3], "\n") 

# check the length of the new dataframe
print("Number of duplicates dropped:", (len(car_details)-len(car_details_clean)))
print("Length of dataframe without duplicates:", len(car_details_clean))

Make and Model       Year  Mileage  Transmission  Engine Size (L)  Fuel Type  Price (£)  Indicated Value
MINI Hatchback       2013  48000    Manual        1.6              Petrol     5500       Good               23
Bentley Continental  2008  55216    Automatic     6.0              Petrol     36200      Good               23
Vauxhall Astra       2007  72000    Manual        1.6              Petrol     1999       Fair               20
dtype: int64 

Make and Model    Year  Mileage  Transmission  Engine Size (L)  Fuel Type  Price (£)  Indicated Value
Volvo v50 Series  2007  124597   Manual        2.0              Diesel     1995       Lower              1
Hyundai I10       2015  47036    Manual        1.0              Petrol     4300       Great              1
                        32095    Automatic     1.2              Petrol     6795       Good               1
dtype: int64 

Number of duplicates dropped: 2934
Length of dataframe without duplicates: 13674


### 3.2. Dropping cars with NaN information

In [23]:
# Replace cells which had "NaN" appended in the scraping to numpy-recognised NaN's
car_details_clean = car_details_clean.replace('NaN', np.nan) # replace NaNs
nan_values = car_details_clean[car_details_clean.isna().any(axis=1)] # Get rows with "NaN" values
print("Number of cars with NaN present:", len(nan_values))
nan_values[:3] # Show the rows with "NaN" values

Number of cars with NaN present: 29


,Make and Model,Year,Mileage,Transmission,Engine Size (L),Fuel Type,Price (£),Indicated Value
5823,Nissan Leaf,2011,74679,NaN,NaN,NaN,4295,Great
7134,BMW 320d,2012,65000,NaN,2.0,NaN,9499,Good
8009,Citroen DS5,2013,101280,Automatic,2.0,NaN,5395,Lower


In [24]:
# Drop the cells which had NaN and reset the index 

car_details_clean.dropna(axis = 0, how = "any", inplace=True)
car_details_clean = car_details_clean.reset_index(drop=True)
print("Length of dataframe without NaN cells:", len(car_details_clean))
car_details_clean[:3]

Length of dataframe without NaN cells: 13645


,Make and Model,Year,Mileage,Transmission,Engine Size (L),Fuel Type,Price (£),Indicated Value
0,Renault Clio,2006,55378,Manual,1.6,Petrol,1675,Great
1,SEAT Leon,2006,110988,Manual,2.0,Petrol,2350,Great
2,MINI Hatch Cooper,2006,69000,Manual,1.6,Petrol,2499,Fair


### 3.3. Finding all car makes and splitting make and model into respective columns

In [26]:
# Finding all potential car makes and cleaning the list

# URL in year range 2006-2020
car_makes_url = "https://www.autotrader.co.uk/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2006&year-to=2020&exclude-writeoff-categories=on&page=1"

# Empty list
car_makes = []

# Function to get all car makes
def get_car_makes(url):
    page = scraper.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    return soup.find("div", class_="sf-flyout__options js-flyout-options").text
car_makes.append(get_car_makes(car_makes_url))

# Clean the list 
# iterate through the uncleaned list of car_makes and split string whenever there was an "\n"
for i, item in enumerate(car_makes):
    car_makes_clean = ' '.join(item.split("\n"))

# remove any non-letters
car_makes_clean = re.sub(r'[^A-z -]+', '', car_makes_clean)

# strip leading and trailing whitespaces and make lowercase
car_makes_clean = car_makes_clean.strip().lower()

#split at three spaces to get a list of the individual makes
car_makes_clean = re.split("   ", car_makes_clean)
print(car_makes_clean)

['abarth', 'aixam', 'ak', 'alfa romeo', 'alpine', 'ariel', 'aston martin', 'audi', 'austin', 'beauford', 'bentley', 'bmw', 'bramwith', 'cadillac', 'caterham', 'chesil', 'chevrolet', 'chrysler', 'citroen', 'corvette', 'cupra', 'dacia', 'daihatsu', 'daimler', 'dax', 'dodge', 'ds automobiles', 'fiat', 'ford', 'fso', 'ginetta', 'gmc', 'great wall', 'holden', 'honda', 'hummer', 'hyundai', 'infiniti', 'isuzu', 'iveco', 'jaguar', 'jeep', 'kia', 'lancia', 'land rover', 'levc', 'lexus', 'lincoln', 'london taxis international', 'lotus', 'mahindra', 'maserati', 'mazda', 'mercedes-benz', 'mg', 'microcar', 'mini', 'mitsubishi', 'morgan', 'nissan', 'opel', 'perodua', 'peugeot', 'pilgrim', 'porsche', 'proton', 'rage', 'renault', 'reva', 'rover', 'saab', 'santana', 'seat', 'sebring', 'skoda', 'smart', 'ssangyong', 'subaru', 'suzuki', 'tesla', 'tiger', 'toyota', 'tvr', 'ultima', 'vauxhall', 'volkswagen', 'volvo', 'westfield', 'yamaha', 'zenos']


In [27]:
# Iterate through the make and model column and when the make and model matches the clean make, append to...
# the make list and then add to the dataframe as a new column

# Make lowercase
car_details_clean["Make and Model"] = car_details_clean["Make and Model"].str.lower()

clean_make_list = []
clean_model_list = []

# For loop which goes through the make and model column and when it finds a match with the clean car makes...
# it appends that to the make list and appends the rest to the model list
for car in car_details_clean["Make and Model"]:
    for make in car_makes_clean:
        if make in car:
            clean_make_list.append(make)
            clean_model_list.append(car[len(make):].strip())
            break # when found, move onto the next car

# Create new columns in the dataframe from the lists
car_details_clean["Make"] = clean_make_list
car_details_clean["Model"] = clean_model_list

# Add the make and model columns to the dataframe in place of the one "Make and Model" column previously
car_details_clean = car_details_clean[['Make', 'Model', 'Year', 'Mileage', 'Transmission', 'Engine Size (L)', 'Fuel Type', 'Price (£)', 'Indicated Value']]
car_details_clean


,Make,Model,Year,Mileage,Transmission,Engine Size (L),Fuel Type,Price (£),Indicated Value
0,renault,clio,2006,55378,Manual,1.6,Petrol,1675,Great
1,seat,leon,2006,110988,Manual,2.0,Petrol,2350,Great
2,mini,hatch cooper,2006,69000,Manual,1.6,Petrol,2499,Fair
3,ford,fiesta,2006,85714,Manual,1.6,Diesel,1699,Great
4,suzuki,grand vitara,2006,77000,Manual,1.6,Petrol,2450,Great
...,...,...,...,...,...,...,...,...,...
13640,kia,sportage,2020,7969,Manual,1.6,Petrol,22779,Great
13641,seat,leon,2020,4943,Manual,1.5,Petrol,18690,Good
13642,volkswagen,golf,2020,26439,Manual,1.5,Petrol,16998,Lower
13643,vauxhall,crossland x,2020,12063,Manual,1.2,Petrol,13349,Good


# 4. SAVE AS CSV

In [28]:
car_details_clean.to_csv("Scraped Cars.csv")